In [1]:
!pip install --no-index ../input/natsort/natsort-8.4.0-py3-none-any.whl --find-links=../input/natsort

Looking in links: ../input/natsort
Processing /kaggle/input/natsort/natsort-8.4.0-py3-none-any.whl


In [2]:
import numpy as np
import pandas as pd
import os
import glob
import pydicom
import cv2
from natsort import natsorted
from statistics import mean

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *
import tensorflow.keras.backend as K

2024-10-13 11:47:35.797588: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 11:47:35.797695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 11:47:35.925552: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
DEBUG = False
if DEBUG == True:
    base_dir = '/kaggle/input/rsna-lsdc-2024-submission-debug-dataset/debug/'
else:
    base_dir = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/'

In [4]:
df = pd.read_csv(base_dir + 'test_series_descriptions.csv')
df = df.astype({'study_id':'str','series_id':'str'})
df

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [5]:
scs_model = keras.models.load_model('/kaggle/input/efficientnet-training-model/scs_model.h5')
nfn_model = keras.models.load_model('/kaggle/input/efficientnet-training-model/nfn_model.h5')
ss_model = keras.models.load_model('/kaggle/input/efficientnet-training-model/ss_model.h5')

extract_scs = keras.models.load_model('/kaggle/input/extraction-spinal-canal-densenet/scs_extraction_model.h5')
extract_nfn = keras.models.load_model('/kaggle/input/extraction-neural-foraminal-densenet/nfn_extraction_model.h5')
extract_ss = keras.models.load_model('/kaggle/input/extraction-subarticular-stenosis-densenet/ss_extraction_model.h5')

In [6]:
def replace_pred(description,box):
    
    if d == 'Sagittal T2/STIR':
        matrix = np.full((5,3),0.3333, dtype=np.float32)
        box.append(matrix)
    else:
        matrix = np.full((10,3),0.3333, dtype=np.float32)
        box.append(matrix)

In [7]:
def SagittalT2_predict(files):

    image = []
    
    for f in files:
        
        dcm = pydicom.dcmread(f)
        dcm = dcm.pixel_array
        dcm = (dcm - dcm.min()) / (dcm.max() - dcm.min() + 1e-6)*255.0
        dcm = dcm / 255.0
        dcm = cv2.resize(dcm,(224,224))
        image.append(dcm)

    pred_image = np.array(image)
    pred = extract_scs.predict(pred_image,verbose=0)
    rank = np.sort(pred[:,1])
    image_slice = np.where(pred[:,1] >= rank[-1])[0].tolist()[0]
    eximage = image[image_slice]

    condition_pred = scs_model.predict(eximage[np.newaxis,:,:,np.newaxis],verbose=0)

    return np.array(condition_pred).squeeze()

In [8]:
def SagittalT1_predict(files):
    
    if len(files) < 4:
        return np.full((10,3),0.33333,dtype=np.float32)
        
    else:
            
        left_images = []
        right_images = []

        average = []

        for f in files:
                
            dcm = pydicom.dcmread(f)
            average.append(dcm[(0x0020, 0x0032)][0])

        means = mean(average)

        for f in files:
                
            dcm = pydicom.dcmread(f)
            position = dcm[(0x0020, 0x0032)][0]
            dcm = dcm.pixel_array
            dcm = (dcm - dcm.min()) / (dcm.max() - dcm.min() + 1e-6)*255.0
            dcm = dcm / 255.0
            dcm = cv2.resize(dcm,(224,224))

            if means < position:
                    
                left_images.append(dcm)

            elif means > position:
                    
                right_images.append(dcm)

        predleft_images = np.array(left_images)[:,:,:,np.newaxis]
        predright_images = np.array(right_images)[:,:,:,np.newaxis]
        left_pred = extract_nfn.predict(predleft_images,verbose=0)
        right_pred = extract_nfn.predict(predright_images,verbose=0)

        left_rank = np.sort(left_pred[:,1])
        left_slice = np.where(left_pred[:,1] >= left_rank[-2])[0].tolist()
        right_rank = np.sort(right_pred[:,1])
        right_slice = np.where(right_pred[:,1] >= right_rank[-2])[0].tolist()

        left_list = []
            
        for ls in left_slice:
                
            im = left_images[ls]
            left_list.append(im)

        right_list = []
            
        for rs in right_slice:
                
            im = right_images[rs]
            right_list.append(im)

        left = np.array(left_list).transpose(1,2,0)[np.newaxis,:,:,:]
        right = np.array(right_list).transpose(1,2,0)[np.newaxis,:,:,:]

        condition_pred = nfn_model.predict([left,right],verbose=0)
        left = condition_pred[0].squeeze()
        right = condition_pred[1].squeeze()

        return np.concatenate((left, right), axis=0)

In [9]:
def AxialT2_predict(files):
    
    if len(files) < 10:
        return np.full((10,3),0.33333,dtype=np.float32)
    
    else:
        
        image = []
        
        for f in files:
            
            dcm = pydicom.dcmread(f)
            dcm = dcm.pixel_array
            dcm = (dcm - dcm.min()) / (dcm.max() - dcm.min() + 1e-6)*255.0
            dcm = dcm / 255.0
            dcm = cv2.resize(dcm,(224,224))
            image.append(dcm)
            
        pred_image = np.array(image)[:,:,:,np.newaxis]
        pred = extract_ss.predict(pred_image,verbose=0)
        rank = np.sort(pred[:,1])
        image_slice = np.where(pred[:,1] >= rank[-10])[0].tolist()

        exim = []
        
        for sl in image_slice:
            
            eximage = image[sl]
            exim.append(eximage)

        exim = np.array(exim).transpose(1,2,0)[np.newaxis,:,:,:]
        condition_pred = ss_model.predict(exim,verbose=0)
        left = condition_pred[0].squeeze()
        right = condition_pred[1].squeeze()
        return np.concatenate((left, right), axis=0)

In [10]:
# sutdy_id 抽出
study_id = df['study_id'].unique()
description_list = ['Sagittal T2/STIR','Sagittal T1','Axial T2']

label_list = ['spinal_canal_stenosis_l1_l2',
              'spinal_canal_stenosis_l2_l3',
              'spinal_canal_stenosis_l3_l4',
              'spinal_canal_stenosis_l4_l5',
              'spinal_canal_stenosis_l5_s1',
              'left_neural_foraminal_narrowing_l1_l2',
              'left_neural_foraminal_narrowing_l2_l3',
              'left_neural_foraminal_narrowing_l3_l4',
              'left_neural_foraminal_narrowing_l4_l5',
              'left_neural_foraminal_narrowing_l5_s1',
              'right_neural_foraminal_narrowing_l1_l2',
              'right_neural_foraminal_narrowing_l2_l3',
              'right_neural_foraminal_narrowing_l3_l4',
              'right_neural_foraminal_narrowing_l4_l5',
              'right_neural_foraminal_narrowing_l5_s1',
              'left_subarticular_stenosis_l1_l2',
              'left_subarticular_stenosis_l2_l3',
              'left_subarticular_stenosis_l3_l4',
              'left_subarticular_stenosis_l4_l5',
              'left_subarticular_stenosis_l5_s1',
              'right_subarticular_stenosis_l1_l2',
              'right_subarticular_stenosis_l2_l3',
              'right_subarticular_stenosis_l3_l4',
              'right_subarticular_stenosis_l4_l5',
              'right_subarticular_stenosis_l5_s1']

lists = []

# study_id を繰り返し
for st in study_id:

    study_df = df[df['study_id']==st]
    study_label = st + '_' + pd.DataFrame(label_list, columns=['row_id'])
    pred_list = []
    # description　を繰り返し
    for d in description_list:
        description = study_df[study_df['series_description']==d]

        # dataframe に該当の description があるかないかで分岐
        if description.shape[0] == 0:
            
            replace_pred(d, pred_list)
            
        else:
            
            nplists = []
            lens = len(description)

            # description の数を繰り返し
            for dl in range(lens):
                
                study = description.iloc[dl,0]
                series = description.iloc[dl,1]
                path = os.path.join(base_dir, 'test_images/', study, series)

                # 該当のディレクトリがあるかないかで分岐
                isdir = os.path.isdir(path)
                
                if isdir == False:

                    replace_pred(d,nplists)
                    
                else:
                    
                    files = natsorted(glob.glob(path + '/*.dcm'))
                    if d == 'Sagittal T2/STIR':
                        sagittalt2_pred = SagittalT2_predict(files)
                        nplists.append(sagittalt2_pred)
                    elif d == 'Sagittal T1':
                        sagittalt1_pred = SagittalT1_predict(files)
                        nplists.append(sagittalt1_pred)
                    elif d == 'Axial T2':
                        axialt2_pred = AxialT2_predict(files)
                        nplists.append(axialt2_pred)
                    
            sums = np.sum(nplists,axis=0)/lens
            pred_list.append(sums)
    predict = pd.DataFrame(np.concatenate(pred_list,axis=0), columns=['normal_mild','moderate','severe'])
    create_df = pd.concat([study_label,predict],axis=1)
    lists.append(create_df)

I0000 00:00:1728820077.884439      71 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [11]:
pd.set_option('display.max_rows', None)
pred = pd.concat(lists,axis=0)
submit = pred.reset_index(drop=True)
submit

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l1_l2,0.989024,0.005901,5.075578e-03
1,44036939_spinal_canal_stenosis_l2_l3,0.368770,0.611340,1.989003e-02
2,44036939_spinal_canal_stenosis_l3_l4,0.240639,0.361855,3.975061e-01
3,44036939_spinal_canal_stenosis_l4_l5,0.019710,0.156412,8.238770e-01
4,44036939_spinal_canal_stenosis_l5_s1,0.913309,0.057345,2.934566e-02
5,44036939_left_neural_foraminal_narrowing_l1_l2,0.999638,0.000355,6.510284e-06
6,44036939_left_neural_foraminal_narrowing_l2_l3,0.999863,0.000133,3.707028e-06
7,44036939_left_neural_foraminal_narrowing_l3_l4,0.994885,0.005085,3.041685e-05
8,44036939_left_neural_foraminal_narrowing_l4_l5,0.895003,0.101249,3.748777e-03
9,44036939_left_neural_foraminal_narrowing_l5_s1,0.461324,0.297713,2.409626e-01


In [12]:
submit.to_csv('/kaggle/working/submission.csv', index=False)